### USCRN Data: High-Octane Scraping

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import yaml 
import re
import itertools

with open ("sources.yaml", "r") as yaml_file:
  sources = yaml.load(yaml_file, Loader=yaml.FullLoader)

##### 1.) Scrape Column Headers and Descriptions 

In [3]:
header_url = sources['USCRN']['headers']
header_response = requests.get(header_url)
header_soup = BeautifulSoup(header_response.content, "html.parser")

columns = str(header_soup).split("\n")[1].strip(" ").split(" ")
columns = list(map(lambda x: str.lower(x), columns)) # columns = [str.lower(c) for c in columns] -- faster?
columns.insert(0,'station_location')

descrip_text = str(header_soup).split("\n")[2] # raw text block containing column descriptions
descrip_text

"The station WBAN number. The UTC date of the observation. The UTC time of the observation. Time is the end of the observed hour, so the 0000 hour is actually the last hour of the previous day's observation (starting just after 11:00 PM through midnight). The Local Standard Time (LST) date of the observation. The Local Standard Time (LST) time of the observation. Time is the end of the observed hour (see UTC_TIME description). The version number of the station datalogger program that was in effect at the time of the observation. Note: This field should be treated as text (i.e. string). Station longitude, using WGS-84. Station latitude, using WGS-84. Average air temperature, in degrees C, during the last 5 minutes of the hour. See Note F. Average air temperature, in degrees C, for the entire hour. See Note F. Maximum air temperature, in degrees C, during the hour. See Note F. Minimum air temperature, in degrees C, during the hour. See Note F. Total amount of precipitation, in mm, record

The descriptions of the columns are quite the mess, as there is no standard separator used. We will have to work our way through it step by step: 

In [4]:
first_split = [re.sub(r'(\([^)]*)$', r"\1)", s) for s in descrip_text.split("). ")] # add ')' back after splitting text on ').' 
no_notes = [re.sub(r' See Note [A-Z]\.',"",s) for s in first_split] # drop any references to notes

The third entry in `no_notes` is ready. The last set of descriptions in `no_notes` can be split on `". "`, but the first two sets need special attention. We will pop the last set out and split it, then pop the third set out, and then address the first two sets. At that point we will recombine everything into one list while preserving the original order. 

In [5]:
last_set = no_notes.pop().strip().split(". ")
third_set = no_notes.pop() # Note: just a string

In [6]:
def flatten(ls:list): 
  return list(itertools.chain.from_iterable(ls)) 

no_notes = [re.sub(". Time is", " at", s) for s in no_notes]
first_second = flatten([s.split(". ") for s in no_notes])

# Finally:
descriptions = flatten([["Location name for USCRN station"], first_second, [third_set], last_set]) # Description added for "station_location" 

In [15]:
header_info = {
  'col_name': columns,
  'description': descriptions, 
  'units': ["X...(Various Lengths)","XXXXX", "YYYYMMDD", "HHmm", "YYYYMMDD", "HHmm", "XXXXXX", "Decimal_degrees", "Decimal_degrees", "Celsius", "Celsius", "Celsius", "Celsius", "mm", "W/m^2", "X", "W/m^2", "X", "W/m^2", "X", "X", "Celsius", "X", "Celsius", "X", "Celsius", "X", "%", "X", "m^3/m^3", "m^3/m^3", "m^3/m^3", "m^3/m^3", "m^3/m^3", "Celsius", "Celsius", "Celsius", "Celsius", "Celsius"]
}

header_df = pd.DataFrame(header_info)
header_df.to_csv("data/column_descriptions.csv", index=True)

##### 2.) Scrape Core Data Files (>2 million rows)

In [17]:
base_url = sources["USCRN"]["index"]
base_response = requests.get(base_url)
base_soup = BeautifulSoup(base_response.content, "html.parser")

In [18]:
links = base_soup.find_all("a") # 'links' in this notebook will refer to <a> elements, not urls
years = [str(x).zfill(1) for x in range(2000,2024)]
year_links = [link for link in links if link['href'].rstrip('/') in years]

file_urls = []
for year_link in year_links: 
  year_url = base_url + year_link.get("href")
  response = requests.get(year_url) 
  soup = BeautifulSoup(response.content, 'html.parser')
  file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
  if file_links:
    new_file_urls = [year_url + link.getText() for link in file_links]
    file_urls.extend(new_file_urls)

In [146]:
rows = []
regex = r"([St.]*[A-Z][a-z]+_*[A-Za-z]*).*.txt" 
for url in file_urls:
  # Get location from url -- will add to BS results in next step
  file_name = re.search(regex, url).group(0)
  station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
  # Get results 
  response = requests.get(url)
  soup = BeautifulSoup(response.content,'html.parser')
  soup_lines = [station_location + " " + line for line in str(soup).split("\n")] # <-- Before modification (see explanation below)
  # soup_lines = [station_location + " " + line for line in str(soup).strip().split("\n")] # <-- Correct code 
  new_rows = [re.split('\s+', row) for row in soup_lines]
  rows.extend(new_rows)

In [148]:
df = pd.DataFrame(rows)

In [24]:
df[df.isna().any(axis=1)]

,station_location,wbanno,utc_date,utc_time,lst_date,lst_time,crx_vn,longitude,latitude,t_calc,...,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
3459,Fairbanks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6941,Utqiagvik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15702,Fairbanks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24463,Utqiagvik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33248,Fairbanks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157609,St._Paul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2158670,Tok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2159731,Toolik_Lake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2160793,Utqiagvik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Are these NaN rows being generated by my code, or are they present in the original data? Note there are 281 NaN rows and 281 urls in file_urls -- this suggests that there is one NaN row being made for each. Let's test the same code on a single url to see what's happening.


In [26]:
test_url = "https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/2003/CRNH0203-2003-AK_Utqiagvik_formerly_Barrow_4_ENE.txt"

file_name = re.search(regex, test_url).group(0)
station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
response = requests.get(test_url)
soup = BeautifulSoup(response.content,'html.parser')
soup_lines = [station_location + " " + line for line in str(soup).split("\n")]
rows = [re.split('\s+', row) for row in soup_lines]

In [46]:
print(rows[-1])
display(pd.DataFrame(data=rows, columns=columns))

['Utqiagvik', '']


,station_location,wbanno,utc_date,utc_time,lst_date,lst_time,crx_vn,longitude,latitude,t_calc,...,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
0,Utqiagvik,27516,20030101,0100,20021231,1600,1.001,-156.61,71.32,-20.9,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,Utqiagvik,27516,20030101,0200,20021231,1700,1.001,-156.61,71.32,-22.5,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,Utqiagvik,27516,20030101,0300,20021231,1800,1.001,-156.61,71.32,-23.9,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,Utqiagvik,27516,20030101,0400,20021231,1900,1.001,-156.61,71.32,-24.9,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,Utqiagvik,27516,20030101,0500,20021231,2000,1.001,-156.61,71.32,-25.7,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,Utqiagvik,27516,20031231,2100,20031231,1200,1.200,-156.61,71.32,-14.7,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
8757,Utqiagvik,27516,20031231,2200,20031231,1300,1.200,-156.61,71.32,-13.8,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
8758,Utqiagvik,27516,20031231,2300,20031231,1400,1.200,-156.61,71.32,-14.9,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
8759,Utqiagvik,27516,20040101,0000,20031231,1500,1.200,-156.61,71.32,-18.2,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


Each text file has a trailing whitespace. We could edit our code to fix this by inserting a `strip()` before splitting the soup object on newlines: 
```python 
[station_location + " " + line for line in str(soup).strip().split("\n")]
```
Since it took almost 20 minutes to successfully download all 2 million+ rows of data, we'll just edit our dataframe rather than re-running that code with the corrected line. 

In [2]:
df.dropna(thresh=2, inplace=True) # station_location is not NaN in those rows. 

Next let's drop the soil temperature and moisture columns. These fields are erroneous (-99.0 or -9999.0) for the vast majority of records.

In [3]:
soil_columns = df.filter(regex="soil")
soil_columns[soil_columns.isin([-99.0, -9999.0]).any(axis=1)]

,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
0,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...
2161849,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2161850,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2161851,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2161852,-99.0,-99.0,-99.0,-99.0,-99.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [4]:
df = df.filter(regex="^((?!soil).)*$")

The `solar_rad` and `sur_temp` sets of columns also have some erroneous measurements, but to a much lesser extent. We will keep these.

In [5]:
st_cols = df.filter(regex="sur_temp")
print(len(st_cols[st_cols.iloc[:,1:].le(-99).any(axis=1)]))

solar_cols = df.filter(regex="solar")
print(len(solar_cols[solar_cols.le(-1).any(axis=1)]))

148770
147377


In [ ]:
# Saving dataframe
df.to_csv("data/uscrn.csv")

Let's also make a table for our various station locations. This will be useful when searching for the four-day forecasts in the NWS notebook. 

In [1]:
# import pandas as pd 
# df = pd.read_csv("data/uscrn.csv")
locations = df[['station_location', 'wbanno', 'longitude', 'latitude']].drop_duplicates()
locations.to_csv("data/locations.csv", index=False)

##### 3.) Upload to BigQuery 

In [79]:
%%bash
bq mk -d --location=us-east4 team-week3:alaska

Dataset 'team-week3:alaska' successfully created.


Core Data: 

In [12]:
%%bash
bq load --source_format=CSV \
  --project_id=team-week3 \
  --autodetect \
  alaska.uscrn data/uscrn.csv

(For now, not uploading `column_descriptions.csv`)